# Морфология 4
В этом ноутбуке описана подготовка данных для задачи POS-tagging. А также пара простых моделей на keras, решающих данную задачу. Оригинальная задача и ноутбук есть в контесте: https://www.kaggle.com/c/rupos2018/overview

## Часть 1. Загрузка корпуса
Здесь мы прочитаем корпуса из csv и разложим их по спискам.

In [3]:
# Имена файлов с данными.
TRAIN_FILENAME = "data/train.csv"
TEST_FILENAME = "data/test.csv"

In [4]:
# Считывание файлов.
from collections import namedtuple
WordForm = namedtuple("WordForm", "word pos gram")

def get_sentences(filename, is_train):
    sentences = []
    with open(filename, "r") as r:
        # Пропускаем заголовок
        next(r)
        sentence = [] # будем заполнять список предложений
        for line in r:
            # предложения отделены по '\n'
            if len(line.strip()) == 0:
                if len(sentence) == 0:
                    continue
                sentences.append(sentence)
                sentence = []
                continue
            if is_train:
                # Формат: индекс\tномер_в_предложении\tсловоформа\tPOS#Грамемы
                word = line.strip().split("\t")[2]
                pos = line.strip().split("\t")[3].split("#")[0]
                gram = line.strip().split("\t")[3].split("#")[1]
                sentence.append(WordForm(word, pos, gram))
            else:
                word = line.strip().split("\t")[2]
                sentence.append(word)
        if len(sentence) != 0:
            sentences.append(sentence)
    return sentences

In [5]:
train = get_sentences(TRAIN_FILENAME, True)
test = get_sentences(TEST_FILENAME, False)

In [6]:
# Выыедем, что получилось
for wordform in train[0][:10]:
    print(wordform.word, '\t', wordform.pos, '\t', wordform.gram)

А 	 CONJ 	 _
ведь 	 PART 	 _
для 	 ADP 	 _
конкретных 	 ADJ 	 Case=Gen|Degree=Pos|Number=Plur
изделий 	 NOUN 	 Animacy=Inan|Case=Gen|Gender=Neut|Number=Plur
зачастую 	 ADV 	 Degree=Pos
нужен 	 ADJ 	 Degree=Pos|Gender=Masc|Number=Sing|Variant=Brev
монокристалл 	 NOUN 	 Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing
не 	 PART 	 _
только 	 PART 	 _


Для простоты далее будем использовать токены слов и POS-теги. Но чтобы определять грамматические значения нужно еще провести некоторые манипуляции с данными, описанные в оригинальном ноутубке. Мы же ограничимся только определением частей речи

## Часть 2. Подготовка эмбеддингов

Обычно в качестве признаков для обучения сеток используются словные эмбеддинги. Для этого можно скачать предобученные и сохранить их в матрицу, где в расположатся векторы эмбеддингах по индексам, соответсвующих слов

In [23]:
#запомним все уникальные слова и POS-теги в корпусе
word_set = set()
pos_set = set()
for sent in train:
    for wordform in sent:
        word_set.add(wordform.word.lower())
        pos_set.add(wordform.pos)

In [24]:
for word in list(word_set)[:10]: 
    print(word, end=', ')
print()
print(pos_set)

княжества, 1941, опровержения, актового, глоток, скромности, уклонялся, играй, отмыва, духов, 
{'VERB', 'ADP', 'X', 'SCONJ', 'NUM', 'ADJ', 'CONJ', 'NOUN', 'PUNCT', 'DET', 'SYM', 'PART', 'AUX', 'INTJ', 'ADV', 'PRON', 'PROPN'}


In [52]:
#Загрузите эмбеддинги c https://nlp.stanford.edu/projects/glove/ или другие, которые вам нравятся и пропишите путь к ним
import numpy as np

word_embeddings_path = 'data/glove.twitter.27B.50d.txt'
word2idx = {}
word_embeddings = []
embedding_size = None
#Загружаем эмбеддинги
with open(word_embeddings_path, 'r') as f_em:
    for line in f_em:
        split = line.strip().split(" ")
        # Совсем короткие строки пропускаем
        if len(split) <= 2:
            continue
        # Встретив первую подходящую строку, фиксируем размер эмбеддингов
        if embedding_size is None:
            embedding_size = len(split) - 1
            # Также нициализируем эмбеддинги для паддингов и неизвестных слов
            word2idx["PADDING_TOKEN"] = len(word2idx)
            word_embeddings.append(np.zeros(embedding_size))

            word2idx["UNKNOWN_TOKEN"] = len(word2idx)
            word_embeddings.append(np.random.uniform(-0.25, 0.25, embedding_size))
        # После этого все эмбеддинги должны быть одинаковой длины
        if len(split) - 1 != embedding_size:
            continue
            
        #Если слова нет в корпусе, то не будем для него запоминать эмбеддинг        
        if (split[0] not in word_set):
            continue

        word_embeddings.append(np.asarray(split[1:], dtype='float32'))
        word2idx[split[0]] = len(word2idx)

word_embeddings = np.array(word_embeddings, dtype='float32')

In [53]:
len(word2idx)

32402

In [54]:
len(word_set & set(word2idx.keys()))

32400

In [55]:
len(word_set)

98880

Как-то эмбеддинги не сильно подходят для данного корпуса поэтому, просто инициализируем рандмно матрицу эмбеддингов при определении сетки. Вам же предлагается все-таки поискать подходящие эмбеддинги и использовать их при обучении.

## Часть 3. Подготовка данных
Теперь нам остается только пронумеровать все слова и POS-теги и можно переходить к обучению сеток.

In [73]:
word_to_index = {'PAD' : 0, 'UNK' : 1}
for word in word_set:
    word_to_index[word] = len(word_to_index)

In [74]:
embs = np.zeros([len(word_to_index), 50])

In [75]:
for i, word in enumerate(word_to_index):
    if word in word2idx:
        embs[i] = word_embeddings[word2idx[word]]

In [77]:
pos_to_index = {}
index_to_pos = {}
for pos in pos_set:
    pos_to_index[pos] = len(pos_to_index)
    index_to_pos[len(index_to_pos)] = pos

In [78]:
# для полносвязной сетки просто захреначим все в один список
data_X = []
data_Y = []
for sent in train:
    for wordform in sent:
        data_X.append(word_to_index[wordform.word.lower()])
        data_Y.append(pos_to_index[wordform.pos])

In [79]:
print(data_X[:10])
print(data_Y[:10])

[596, 74285, 94677, 55084, 58994, 70371, 92183, 26415, 87672, 2463]
[6, 11, 1, 5, 7, 14, 5, 7, 11, 11]


## Часть 4. Полносвязная сеть
Самой простой моделью является обычный перцептрон. На вход сетки будем подавать просто эмдеддинг каждого слова, на выходе ожидать распредедение вероятностей по тегам. В качестве фреймворка достаточно будет использовать keras и его Sequential модель (https://keras.io/models/sequential/), в которую слои добавляются последовательно, с помощью метода `add`.

In [80]:
from keras.models import Sequential
from keras.layers import Embedding, Dense, Activation, Flatten

In [218]:
model = Sequential()
# на самом деле на вход сетки будет добавляться индекс слова, а слой эмбеддинга будет возвращать для него вектор
model.add(Embedding(input_length=1, input_dim=len(word_to_index), output_dim=50, embeddings_initializer='random_uniform',
                    )) # матрицу эмбеддингов просто инициализируем нормальным распределением и отключим обучение
# далее нам нужно схлопнуть трехмерный тензор с одной фиктивной размерностью в двумерный
model.add(Flatten())
model.add(Dense(100)) # основной полносвязный слой
model.add(Activation('relu')) # для приличия добавим функцию активации
model.add(Dense(len(pos_to_index))) # выходной слой тоже полносвязный размерности по кол-ву тегов
model.add(Activation('softmax')) # ну и в конце делаем softmax, чтобы получить распределение
model.summary() # вывод получившейся модели

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 1, 50)             4944100   
_________________________________________________________________
flatten_7 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 100)               5100      
_________________________________________________________________
activation_14 (Activation)   (None, 100)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 17)                1717      
_________________________________________________________________
activation_15 (Activation)   (None, 17)                0         
Total params: 4,950,917
Trainable params: 4,950,917
Non-trainable params: 0
____________________________________________

In [219]:
model.layers[0].set_weights([embs])

In [220]:
from sklearn.model_selection import train_test_split

In [221]:
data_X_train, data_X_test, data_Y_train, data_Y_test = train_test_split(data_X, data_Y, test_size=0.25)

In [222]:
# компилируем модель
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [223]:
# и обучаем
model.fit(
    np.array(data_X_train),
    np.array(data_Y_train),
    validation_data=(np.array(data_X_test), np.array(data_Y_test)), 
    epochs=1, 
    batch_size=256
)

2493/2493 [==============================] - 82s 33ms/step - loss: 0.4415 - accuracy: 0.8481 - val_loss: 0.2448 - val_accuracy: 0.9073


Проверка обученности модели остается за вами. Этот пример лишь для того, чтобы показать как собрать сетку и скормить ей данные.

## Часть 5. Рекуррентая сеть.

Далее рассмотрим более приближенную к SOTA модель. Ей является рекуррентая сеть, которая принимает эмбеддинги слов в предложении и генерирует для них распределение вероятностей. Основным отличием от прошлой в том, что теперь мы будем использовать соседние слова как раз за счет рекуррентого слоя. Для этой модели мы уже будем использовать функциональный способ задания модели все того же кераса (https://keras.io/models/model/).

In [252]:
from keras.layers import LSTM, TimeDistributed,Bidirectional, Input, Masking
from keras.models import Model

In [292]:
# В начале задается входной слой, в котором мы укажем входную размерность. 
# Это будет None, т.к. мы заранее не знаем, какой будет длина каждого предложения 
input_layer = Input(shape=(None,), name='input')
# Далее идет все тот же слой эмеддинга, которому мы на вход подаем предыдущий слой (в этом и суть functional APO)
embeddings_layer = Embedding(input_dim=len(word_to_index), output_dim=50, 
                             embeddings_initializer='random_uniform',
                             name='embedding',
                             mask_zero=True,
                            )(input_layer)

# Итак, основным слоем здесь будет двусторонний LSTM, который будет возвращать вектор для каждого слова (return_sequences=True) 
blstm_layer = Bidirectional(LSTM(100, return_sequences=True), name='blstm')(embeddings_layer)
# Аналогично т.к. у нас здесь вектора для каждого слоя, то и полносвязный слой должен применяться для каждого слоя 
# по-отдельности. Поэтому полносвязный слой оборачивается в  TimeDistributed
result_layer = TimeDistributed(Dense(len(pos_to_index), activation='softmax', name='result'))(blstm_layer)
# собственно определяем модель входными и выходными слоями
model = Model(inputs=[input_layer], outputs=result_layer)
# компилируем
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# выводим архитектуру
model.summary()

Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 50)          4944100   
_________________________________________________________________
blstm (Bidirectional)        (None, None, 200)         120800    
_________________________________________________________________
time_distributed_7 (TimeDist (None, None, 17)          3417      
Total params: 5,068,317
Trainable params: 5,068,317
Non-trainable params: 0
_________________________________________________________________


In [293]:
model.layers[1].set_weights([embs])

Далее нам нужно было бы распределить слова по предложениям, распределить по группам по длине, выравнить предложения по длине в одной групе, заполнив недостающие слова паддингами. Но это довольно неприятный процесс, а мне просто хочется запустить сетку и проверить, что она вообще работает, что сошлись все разверности. Поэтому просто раскидаем по 10 слов с помощью `numpy.reshape`

In [294]:
dot_idx = word_to_index['.']

In [295]:
sentences = []
ys = []
cur_sent = []
cur_ys = []
for x, y in zip(data_X, data_Y):
    if x == dot_idx:
        sentences.append(np.array(cur_sent))
        ys.append(np.array(cur_ys).reshape(-1, 1))
        cur_sent = []
        cur_ys = []
    else:
        cur_sent.append(x)
        cur_ys.append(y)

In [296]:
len(sentences)

46100

In [298]:
from keras.preprocessing.sequence import pad_sequences
import keras.backend as K

In [299]:
sentences_train, sentences_test, ys_train, ys_test = train_test_split(sentences, ys)

In [300]:
def get_batched_data(sentences, ys, batch_size=10):
    num_batches = len(sentences) // batch_size
    result_X = []
    result_Y = []
    for i in range(num_batches):
        x_batch = pad_sequences(sentences[i * batch_size : (i + 1) * batch_size])
        y_batch  = pad_sequences(ys[i * batch_size : (i + 1) * batch_size])
        yield x_batch, y_batch

In [301]:
model.fit(
    get_batched_data(sentences_train, ys_train), 
    validation_data=get_batched_data(sentences_test, ys_test),
    epochs=1
)

3457/3457 [==============================] - 182s 53ms/step - loss: 0.1754 - accuracy: 0.1214 - val_loss: 0.0874 - val_accuracy: 0.1200


## Плохая точность, потому что я так и не понял, как заставить керас игнорировать паддинги при подсчёте метрик кроме как писать кастомную метрику, так что доверимся лоссу

## Часть 6. Задание
В качестве упражения предлагается довести до ума обучения второй модели: распределить слова по предложениям, написать тестирование модели и собственно посмотреть как оно обучилось. Тестировать предлагаю на последней 1000 предложений, обучать - на остальном. Кто уверен в своих желаниях, то может решить оригинальную задачу: предсказывать также грамматические категории. А мы же перейдем на следующем семинаре к более приятному фреймворку - PyTorch. 